In [2]:
!pip install torch torchvision pillow

In [4]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Hemocytometer DETECT + COUNT (모든 라벨 -> 'cell'), RetinaNet(ResNet50-FPN), NO argparse, FOLDER 입력.
- 입력: Roboflow TensorFlow export를 이미 풀어둔 폴더
        DATASET/
          ├─ train/_annotations.csv  + 이미지들
          └─ valid/_annotations.csv  + 이미지들
- 모델: torchvision RetinaNet ResNet50 FPN (Focal Loss + SmoothL1)
- 출력: OUT_DIR/
        ├─ models/best.pt
        ├─ logs.txt
        ├─ viz_val/*.png   (박스 + gt/pred 총계)
        ├─ viz_test/*.png
        ├─ report_val.csv  (image, gt_count, pred_count)
        └─ report_test.csv
"""

# =======================
# CONFIG (필요시 수정)
# =======================
CONFIG = {
    "IMAGE_SIZE": 640,       # 704/768로 올리면 약간↑(속도↓)
    "BATCH": 6,              # RetinaNet은 메모리 더 씀 → 기존보다 조금 낮춤
    "EPOCHS": 30,
    "LR": 2e-4,              # SSD보다 살짝 낮춤(안정)
    "WD": 5e-4,
    "CONF_THRESH": 0.25,
    "NMS_IOU": 0.5,
    "IOU_MATCH": 0.50,       # eval 매칭 임계
    "USE_CPU": False,
    "SEED": 1337,
    "VAL_TEST_SPLIT": 0.5,   # valid를 val/test 반반
}

# =======================
# Imports
# =======================
import os, io, csv, random, time
from pathlib import Path
from collections import defaultdict

import numpy as np
from PIL import Image, ImageDraw, ImageFont

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import torchvision
from torchvision.transforms.functional import to_tensor
from torchvision.ops import nms, box_iou

# RetinaNet 헤드 (버전별 안전 교체용)
try:
    from torchvision.models.detection.retinanet import RetinaNetClassificationHead
except Exception:
    RetinaNetClassificationHead = None

import cv2 as cv

# =======================
# Utils
# =======================
def set_seed(s=1337):
    random.seed(s); np.random.seed(s); torch.manual_seed(s); torch.cuda.manual_seed_all(s)

def ensure_dir(p):
    Path(p).mkdir(parents=True, exist_ok=True)

def log(msg, fp=None):
    print(msg, flush=True)
    if fp: fp.write(msg + "\n"); fp.flush()

def load_font():
    try:
        return ImageFont.truetype("arial.ttf", 18)
    except:
        try:
            return ImageFont.truetype("/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf", 18)
        except:
            return ImageFont.load_default()

def _csv_value(row, *keys, default=None):
    for k in keys:
        if k in row and row[k] != "":
            return row[k]
    return default

# =======================
# Preprocess (grayscale + CLAHE + background suppression)
# =======================
def preprocess_for_detector(pil_img, size):
    """
    RGB -> Gray -> CLAHE -> 큰 커널 열림(배경/격자) -> 감산 -> 0..255 정규화
    -> 그레이 3채널 스택 -> model input size 리사이즈 -> PIL RGB 반환
    """
    im = np.array(pil_img)                    # RGB uint8
    g  = cv.cvtColor(im, cv.COLOR_RGB2GRAY)
    clahe = cv.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    g  = clahe.apply(g)
    se = cv.getStructuringElement(cv.MORPH_RECT, (41,41))
    bg = cv.morphologyEx(g, cv.MORPH_OPEN, se)
    g  = cv.subtract(g, bg)
    g  = cv.normalize(g, None, 0, 255, cv.NORM_MINMAX)
    im3 = np.stack([g, g, g], axis=2)                    # 3ch
    im3 = cv.resize(im3, (size, size), cv.INTER_LINEAR)  # resize
    return Image.fromarray(im3, mode="RGB")

# =======================
# Data Loading (from DIR)
# =======================
def read_split_from_dir(base_dir, split):  # split in {"train","valid"}
    base_dir = Path(base_dir)
    csv_path = base_dir / split / "_annotations.csv"
    if not csv_path.exists():
        raise FileNotFoundError(f"{csv_path} not found")

    by_file = defaultdict(list)
    with open(csv_path, "r", encoding="utf-8") as f:
        rows = csv.DictReader(f)
        for r in rows:
            fn = _csv_value(r, "filename", "file", "image")
            if not fn:
                continue
            W = int(float(_csv_value(r, "width", default="0")))
            H = int(float(_csv_value(r, "height", default="0")))
            xmin = float(_csv_value(r, "xmin", default="0"))
            ymin = float(_csv_value(r, "ymin", default="0"))
            xmax = float(_csv_value(r, "xmax", default="0"))
            ymax = float(_csv_value(r, "ymax", default="0"))
            by_file[fn].append((W, H, xmin, ymin, xmax, ymax))

    items = []
    for fn, rows_ in by_file.items():
        img_path = base_dir / split / fn
        if not img_path.exists():
            cand = None
            for p in (base_dir / split).glob(f"**/{Path(fn).name}"):
                cand = p; break
            if cand is None:
                bn = Path(fn).stem
                for p in (base_dir / split).glob(f"**/{bn}.*"):
                    cand = p; break
            if cand is None:
                continue
            img_path = cand

        W = int(rows_[0][0]); H = int(rows_[0][1])
        boxes = []
        for (_, _, x1, y1, x2, y2) in rows_:
            x1 = max(0.0, min(float(x1), W-1))
            y1 = max(0.0, min(float(y1), H-1))
            x2 = max(0.0, min(float(x2), W-1))
            y2 = max(0.0, min(float(y2), H-1))
            if x2 > x1 and y2 > y1:
                boxes.append([x1, y1, x2, y2])

        items.append({
            "path": str(img_path.resolve()),
            "W": W, "H": H,
            "boxes": boxes,
            "labels": [1]*len(boxes),  # 단일 클래스 'cell'
        })
    return items

def split_valid_into_val_test(valid_items, test_ratio=0.5, seed=1337):
    rnd = random.Random(seed)
    idx = list(range(len(valid_items)))
    rnd.shuffle(idx)
    k = int(len(idx) * test_ratio)
    test_idx = set(idx[:k])
    val, test = [], []
    for i, it in enumerate(valid_items):
        (test if i in test_idx else val).append(it)
    return val, test

# =======================
# Dataset / Collate
# =======================
class DetectDataset(Dataset):
    def __init__(self, items, img_size=640, train=True, hflip_p=0.5, vflip_p=0.0):
        self.items = items
        self.size = img_size
        self.train = train
        self.hflip_p = hflip_p
        self.vflip_p = vflip_p

    def __len__(self): return len(self.items)

    def __getitem__(self, i):
        it = self.items[i]
        im = Image.open(it["path"]).convert("RGB")
        W, H = im.size

        # grayscale 기반 전처리 + resize
        im = preprocess_for_detector(im, self.size)

        # 박스 resize (원본->self.size로 선형 스케일)
        sx = self.size / W; sy = self.size / H
        boxes = [[x1*sx, y1*sy, x2*sx, y2*sy] for x1,y1,x2,y2 in it["boxes"]]

        # 간단 aug
        if self.train:
            if random.random() < self.hflip_p:
                im = im.transpose(Image.FLIP_LEFT_RIGHT)
                boxes = [[self.size-x2, y1, self.size-x1, y2] for x1,y1,x2,y2 in boxes]
            if random.random() < self.vflip_p:
                im = im.transpose(Image.FLIP_TOP_BOTTOM)
                boxes = [[x1, self.size-y2, x2, self.size-y1] for x1,y1,x2,y2 in boxes]

        img_t = to_tensor(im)  # [0,1], CxHxW
        target = {}
        if len(boxes):
            target["boxes"] = torch.tensor(boxes, dtype=torch.float32)
            target["labels"] = torch.ones((len(boxes),), dtype=torch.int64)  # class id 1
        else:
            target["boxes"] = torch.zeros((0,4), dtype=torch.float32)
            target["labels"] = torch.zeros((0,), dtype=torch.int64)
        target["image_id"] = torch.tensor([i])
        return img_t, target

def collate_fn(batch):
    imgs, targets = list(zip(*batch))
    return list(imgs), list(targets)

# =======================
# Model (RetinaNet ResNet50-FPN, num_classes=2(background+cell))
# =======================
def build_model(num_classes=2):
    """
    우선 가중치 로드 + num_classes 지정 시도 → 실패하면 헤드 수동 교체.
    """
    # 1) 가중치 API 우선
    try:
        from torchvision.models.detection import (
            retinanet_resnet50_fpn,
            RetinaNet_ResNet50_FPN_Weights,
        )
        model = retinanet_resnet50_fpn(
            weights=RetinaNet_ResNet50_FPN_Weights.COCO_V1,
            num_classes=num_classes,   # 최신 버전에선 헤드가 자동 교체됨
        )
        return model
    except Exception:
        pass

    # 2) 구버전: weights 로드 후 classification head만 교체
    try:
        from torchvision.models.detection import (
            retinanet_resnet50_fpn,
            RetinaNet_ResNet50_FPN_Weights,
        )
        model = retinanet_resnet50_fpn(
            weights=RetinaNet_ResNet50_FPN_Weights.COCO_V1
        )
    except Exception:
        # 아주 구버전 대응
        model = torchvision.models.detection.retinanet_resnet50_fpn(pretrained=True)

    # head 교체 (ClassificationHead만 교체해 num_classes 맞추기)
    try:
        in_channels = model.backbone.out_channels  # FPN out_channels (보통 256)
    except Exception:
        # 안전빵: FPN에 더미 흘려서 채널 수 얻기
        model.eval()
        with torch.no_grad():
            x = torch.zeros(1,3,CONFIG["IMAGE_SIZE"],CONFIG["IMAGE_SIZE"])
            feats = model.backbone(x)  # dict of pyramid maps
            # 아무 레벨 텐서의 채널 수
            in_channels = list(feats.values())[0].shape[1]

    # anchor 개수(레벨마다 동일 가정)
    try:
        num_anchors = model.head.classification_head.num_anchors
    except Exception:
        num_anchors = model.anchor_generator.num_anchors_per_location()[0]

    if RetinaNetClassificationHead is None:
        raise RuntimeError("Your torchvision version lacks RetinaNetClassificationHead; please upgrade torchvision.")

    model.head.classification_head = RetinaNetClassificationHead(
        in_channels=in_channels,
        num_anchors=num_anchors,
        num_classes=num_classes
    )
    return model

# =======================
# Evaluation (박스/카운트)
# =======================
@torch.no_grad()
def simple_eval(model, loader, device, score_thresh=0.25, nms_iou=0.5, iou_thresh=0.5, viz_dir=None, names=None):
    font = load_font()
    ensure_dir(viz_dir) if viz_dir else None

    tot_tp=tot_fp=tot_fn=0
    abs_err=[]

    for bidx, (imgs, targets) in enumerate(loader):
        imgs = [im.to(device) for im in imgs]
        outs = model(imgs)

        for i, (pred, gt) in enumerate(zip(outs, targets)):
            boxes = pred["boxes"].detach().cpu()
            scores = pred["scores"].detach().cpu()
            keep = scores >= score_thresh
            boxes = boxes[keep]; scores = scores[keep]
            if boxes.numel():
                keep_idx = nms(boxes, scores, nms_iou)
                boxes = boxes[keep_idx]
                scores = scores[keep_idx]

            gt_boxes = gt["boxes"].cpu()

            # greedy IoU match
            if len(boxes)>0 and len(gt_boxes)>0:
                ious = box_iou(boxes, gt_boxes)
                matched_p=set(); matched_g=set()
                while True:
                    v, _ = torch.max(ious, dim=1)
                    p = int(torch.argmax(v))
                    g = int(torch.argmax(ious[p]))
                    if p in matched_p or g in matched_g or ious[p,g] < iou_thresh:
                        break
                    matched_p.add(p); matched_g.add(g)
                    ious[p,:]=0; ious[:,g]=0
                tp = len(matched_p)
                fp = len(boxes) - tp
                fn = len(gt_boxes) - tp
            else:
                tp = 0; fp = int(len(boxes)); fn = int(len(gt_boxes))

            tot_tp += tp; tot_fp += fp; tot_fn += fn

            pred_count = int(len(boxes))
            gt_count = int(len(gt_boxes))
            abs_err.append(abs(pred_count - gt_count))

            # viz (전처리된 입력을 그린다)
            if viz_dir:
                im = (imgs[i].cpu().numpy().transpose(1,2,0)*255).astype(np.uint8)
                pil = Image.fromarray(im)
                draw = ImageDraw.Draw(pil)
                for b in boxes:
                    x1,y1,x2,y2 = [float(x) for x in b]
                    draw.rectangle([x1,y1,x2,y2], outline=(255,0,255), width=2)
                txt = f"gt:{gt_count}  pred:{pred_count}"
                draw.rectangle([6,6,6+220,6+26], fill=(0,0,0))
                draw.text((10,8), txt, fill=(255,255,255), font=font)
                name = names[bidx] if names and bidx < len(names) else f"img{bidx:06d}"
                ensure_dir(viz_dir)
                pil.save(Path(viz_dir)/f"viz_{name}_{i:06d}.png")

    prec = tot_tp / max(1, tot_tp + tot_fp)
    rec  = tot_tp / max(1, tot_tp + tot_fn)
    f1   = 2*prec*rec / max(1e-9, (prec+rec))
    mae  = float(np.mean(abs_err)) if len(abs_err) else float("nan")
    return {"precision":prec, "recall":rec, "f1":f1, "mae":mae}

# =======================
# Train / Test
# =======================
def main():
    # ---- 입력 (고정 경로: Kaggle 예시) ----
    base_dir = '/kaggle/input/hemocytomer'   # 데이터셋 폴더
    if not base_dir:
        raise SystemExit("Folder path required.")
    out_dir = '/kaggle/working/'             # 출력 폴더

    C = CONFIG
    set_seed(C["SEED"])
    device = torch.device("cpu" if C["USE_CPU"] or not torch.cuda.is_available() else "cuda")

    out_dir = Path(out_dir); ensure_dir(out_dir); ensure_dir(out_dir/"models")
    logf = open(out_dir/"logs.txt", "w", encoding="utf-8")
    log(f"Device: {device}", logf)
    log(f"Base dir: {base_dir}", logf)

    # 데이터 로드
    train_items = read_split_from_dir(base_dir, "train")
    valid_items = read_split_from_dir(base_dir, "valid")
    val_items, test_items = split_valid_into_val_test(valid_items, test_ratio=C["VAL_TEST_SPLIT"], seed=C["SEED"])
    log(f"Train {len(train_items)}  Val {len(val_items)}  Test {len(test_items)}", logf)

    ds_tr = DetectDataset(train_items, img_size=C["IMAGE_SIZE"], train=True,  hflip_p=0.5, vflip_p=0.0)
    ds_va = DetectDataset(val_items,   img_size=C["IMAGE_SIZE"], train=False)
    ds_te = DetectDataset(test_items,  img_size=C["IMAGE_SIZE"], train=False)

    dl_tr = DataLoader(ds_tr, batch_size=C["BATCH"], shuffle=True,  num_workers=2, collate_fn=collate_fn)
    dl_va = DataLoader(ds_va, batch_size=C["BATCH"], shuffle=False, num_workers=2, collate_fn=collate_fn)
    dl_te = DataLoader(ds_te, batch_size=C["BATCH"], shuffle=False, num_workers=2, collate_fn=collate_fn)

    # 모델 (RetinaNet)
    model = build_model(num_classes=2).to(device)

    # 옵티마이저 (RetinaNet은 FocalLoss라 약간 낮은 LR 권장)
    opt = optim.AdamW([p for p in model.parameters() if p.requires_grad], lr=C["LR"], weight_decay=C["WD"])

    best_f1 = -1.0
    best_file = out_dir/"models"/"best.pt"

    # ---- 학습 ----
    for ep in range(1, C["EPOCHS"]+1):
        model.train()
        loss_sum = 0.0; n=0
        t0 = time.time()
        for imgs, targets in dl_tr:
            imgs = [im.to(device) for im in imgs]
            tgts = [{k:(v.to(device) if torch.is_tensor(v) else v) for k,v in t.items()} for t in targets]
            losses = model(imgs, tgts)      # dict of losses (focal + smoothL1)
            loss = sum(v for v in losses.values())
            opt.zero_grad(); loss.backward(); opt.step()
            loss_sum += float(loss.item()); n += 1
        train_loss = loss_sum / max(1,n)
        dt = time.time()-t0

        # ---- 검증 ----
        model.eval()
        val_metrics = simple_eval(
            model, dl_va, device,
            score_thresh=C["CONF_THRESH"], nms_iou=C["NMS_IOU"], iou_thresh=C["IOU_MATCH"],
            viz_dir=str(out_dir/"viz_val"),
            names=[Path(it["path"]).name for it in val_items],
        )
        log(f"[{ep:03d}] loss={train_loss:.4f}  val_F1={val_metrics['f1']:.3f}  val_MAE={val_metrics['mae']:.2f}  ({dt:.1f}s)", logf)

        if val_metrics["f1"] > best_f1:
            best_f1 = val_metrics["f1"]
            torch.save({"epoch": ep, "model": model.state_dict(), "val": val_metrics}, best_file)
            log(f"  saved best -> {best_file}", logf)

    # ---- 테스트 ----
    ckpt = torch.load(best_file, map_location=device)
    model.load_state_dict(ckpt["model"]); model.eval()

    val_metrics = simple_eval(
        model, dl_va, device,
        score_thresh=C["CONF_THRESH"], nms_iou=C["NMS_IOU"], iou_thresh=C["IOU_MATCH"],
        viz_dir=str(out_dir/"viz_val"),
        names=[Path(it["path"]).name for it in val_items],
    )
    test_metrics = simple_eval(
        model, dl_te, device,
        score_thresh=C["CONF_THRESH"], nms_iou=C["NMS_IOU"], iou_thresh=C["IOU_MATCH"],
        viz_dir=str(out_dir/"viz_test"),
        names=[Path(it["path"]).name for it in test_items],
    )
    log(f"FINAL  Val: P={val_metrics['precision']:.3f} R={val_metrics['recall']:.3f} F1={val_metrics['f1']:.3f} MAE={val_metrics['mae']:.2f}", logf)
    log(f"FINAL Test: P={test_metrics['precision']:.3f} R={test_metrics['recall']:.3f} F1={test_metrics['f1']:.3f} MAE={test_metrics['mae']:.2f}", logf)

    # ---- per-image count CSV ----
    def dump_counts(loader, items, out_csv):
        ensure_dir(Path(out_csv).parent)
        with torch.no_grad(), open(out_csv, "w", encoding="utf-8", newline="") as f:
            w = csv.writer(f); w.writerow(["image", "gt_count", "pred_count"])
            for (imgs, targets), it in zip(loader, items):
                im = imgs[0].to(device)
                pred = model([im])[0]
                boxes = pred["boxes"].detach().cpu()
                scores = pred["scores"].detach().cpu()
                keep = scores >= CONFIG["CONF_THRESH"]
                boxes = boxes[keep]; scores = scores[keep]
                if boxes.numel():
                    k = nms(boxes, scores, CONFIG["NMS_IOU"])
                    boxes = boxes[k]
                w.writerow([it["path"], len(it["boxes"]), int(len(boxes))])

    dump_counts(dl_va, val_items,  out_dir/"report_val.csv")
    dump_counts(dl_te, test_items, out_dir/"report_test.csv")

    log("Done.", logf); logf.close()

if __name__ == "__main__":
    main()


Device: cuda
Base dir: /kaggle/input/hemocytomer
Train 160  Val 20  Test 20


Downloading: "https://download.pytorch.org/models/retinanet_resnet50_fpn_coco-eeacb38b.pth" to /root/.cache/torch/hub/checkpoints/retinanet_resnet50_fpn_coco-eeacb38b.pth
100%|██████████| 130M/130M [00:00<00:00, 218MB/s] 
/tmp/ipykernel_46/1733075914.py:109: DeprecationWarning: 'mode' parameter is deprecated and will be removed in Pillow 13 (2026-10-15)
  return Image.fromarray(im3, mode="RGB")
/tmp/ipykernel_46/1733075914.py:109: DeprecationWarning: 'mode' parameter is deprecated and will be removed in Pillow 13 (2026-10-15)
  return Image.fromarray(im3, mode="RGB")
/tmp/ipykernel_46/1733075914.py:109: DeprecationWarning: 'mode' parameter is deprecated and will be removed in Pillow 13 (2026-10-15)
  return Image.fromarray(im3, mode="RGB")
/tmp/ipykernel_46/1733075914.py:109: DeprecationWarning: 'mode' parameter is deprecated and will be removed in Pillow 13 (2026-10-15)
  return Image.fromarray(im3, mode="RGB")


[001] loss=1.4273  val_F1=0.000  val_MAE=7.10  (31.0s)
  saved best -> /kaggle/working/models/best.pt


/tmp/ipykernel_46/1733075914.py:109: DeprecationWarning: 'mode' parameter is deprecated and will be removed in Pillow 13 (2026-10-15)
  return Image.fromarray(im3, mode="RGB")
/tmp/ipykernel_46/1733075914.py:109: DeprecationWarning: 'mode' parameter is deprecated and will be removed in Pillow 13 (2026-10-15)
  return Image.fromarray(im3, mode="RGB")
/tmp/ipykernel_46/1733075914.py:109: DeprecationWarning: 'mode' parameter is deprecated and will be removed in Pillow 13 (2026-10-15)
  return Image.fromarray(im3, mode="RGB")
/tmp/ipykernel_46/1733075914.py:109: DeprecationWarning: 'mode' parameter is deprecated and will be removed in Pillow 13 (2026-10-15)
  return Image.fromarray(im3, mode="RGB")


[002] loss=1.0078  val_F1=0.027  val_MAE=6.70  (33.0s)
  saved best -> /kaggle/working/models/best.pt


...

[030] loss=0.1011  val_F1=0.911  val_MAE=1.15  (33.5s)


/tmp/ipykernel_46/1733075914.py:109: DeprecationWarning: 'mode' parameter is deprecated and will be removed in Pillow 13 (2026-10-15)
  return Image.fromarray(im3, mode="RGB")
/tmp/ipykernel_46/1733075914.py:109: DeprecationWarning: 'mode' parameter is deprecated and will be removed in Pillow 13 (2026-10-15)
  return Image.fromarray(im3, mode="RGB")
/tmp/ipykernel_46/1733075914.py:109: DeprecationWarning: 'mode' parameter is deprecated and will be removed in Pillow 13 (2026-10-15)
  return Image.fromarray(im3, mode="RGB")
/tmp/ipykernel_46/1733075914.py:109: DeprecationWarning: 'mode' parameter is deprecated and will be removed in Pillow 13 (2026-10-15)
  return Image.fromarray(im3, mode="RGB")


FINAL  Val: P=0.880 R=0.979 F1=0.927 MAE=0.90
FINAL Test: P=0.908 R=0.986 F1=0.946 MAE=0.60


/tmp/ipykernel_46/1733075914.py:109: DeprecationWarning: 'mode' parameter is deprecated and will be removed in Pillow 13 (2026-10-15)
  return Image.fromarray(im3, mode="RGB")
/tmp/ipykernel_46/1733075914.py:109: DeprecationWarning: 'mode' parameter is deprecated and will be removed in Pillow 13 (2026-10-15)
  return Image.fromarray(im3, mode="RGB")
/tmp/ipykernel_46/1733075914.py:109: DeprecationWarning: 'mode' parameter is deprecated and will be removed in Pillow 13 (2026-10-15)
  return Image.fromarray(im3, mode="RGB")
/tmp/ipykernel_46/1733075914.py:109: DeprecationWarning: 'mode' parameter is deprecated and will be removed in Pillow 13 (2026-10-15)
  return Image.fromarray(im3, mode="RGB")


Done.
